<a href="https://colab.research.google.com/github/greyhound101/IEEE-CIS-Fraud/blob/master/DSB2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import os
os.environ['KAGGLE_USERNAME'] = "tapaskd123" # username from the json file
os.environ['KAGGLE_KEY'] = "aba8dc1f085221111d925003fe5a88ed" # key from the json file
!kaggle competitions download -c data-science-bowl-2019

  0% 0.00/10.8k [00:00<?, ?B/s]
100% 10.8k/10.8k [00:00<00:00, 19.4MB/s]
  0% 0.00/262k [00:00<?, ?B/s]
100% 262k/262k [00:00<00:00, 86.6MB/s]
 97% 383M/397M [00:03<00:00, 135MB/s]
100% 397M/397M [00:03<00:00, 118MB/s]
  0% 0.00/399k [00:00<?, ?B/s]
100% 399k/399k [00:00<00:00, 55.8MB/s]
 54% 22.0M/40.8M [00:00<00:00, 46.6MB/s]
100% 40.8M/40.8M [00:00<00:00, 117MB/s] 


Read get starting point and get count of event code for each starting point

In [128]:
%%writefile code_counts.py
from tqdm import tqdm
import gc
import pandas as pd
def create_code_count(df):
  df=pd.read_csv(df+'.csv.zip',usecols=['timestamp','installation_id','event_code'],nrows=100000)
  df['start']=(df['event_code']==2000).astype(int)
  df['uid']=df['start'].cumsum()
  df['timestamp']=pd.to_datetime(df['timestamp'])
  agg=df.groupby(['uid','event_code']).agg({'timestamp':'count'})
  agg=agg.reset_index()
  agg.columns=['uid','event_code','count']
  del df
  gc.collect()
  dcts=[]
  for gp,tmp in tqdm(agg.groupby(['uid'])):
    dk={'uid':gp}
    tmp=tmp.set_index('event_code')
    tmp=tmp['count'].to_dict()
    for uid in agg['event_code'].unique():
      dk['event_code_'+str(uid)]=tmp.get(uid,0)
    dcts.append(dk)
  pd.DataFrame(dcts).to_csv('count.csv',index=False)

Overwriting code_counts.py


In [129]:
%%writefile create_train.py
from code_counts import create_code_count
create_code_count('train')

Overwriting create_train.py


In [130]:
%%writefile create_test.py
from code_counts import create_code_count
create_code_count('test')

Overwriting create_test.py


In [131]:
%%time
!python create_train.py
!python create_test.py

100% 2855/2855 [00:02<00:00, 1019.69it/s]
100% 2663/2663 [00:02<00:00, 1033.29it/s]
CPU times: user 84.2 ms, sys: 28.4 ms, total: 113 ms
Wall time: 11.7 s


Correct or not columns

In [136]:
%%writefile preprocessing.py
import pickle
import pandas as pd
import numpy as np
def load_csv(check):
  df=pd.read_csv(check+'.csv.zip',nrows=100000)
  
  #correct incorrect
  df['correct']=df['event_data'].str.contains('correct:true').astype(int)
  df['incorrect']=df['event_data'].str.contains('correct:flase').astype(int)
  df['attempt']=(((df['event_code']==4100)&(df['title']!='Bird Measurer (Assessment)'))|((df['event_code']==4110)&(df['title']=='Bird Measurer (Assessment)'))).astype(int)
  df=df.drop(['event_data'],1)
  
  
  df['correct']=df['attempt']*df['correct']
  df['incorrect']=df['attempt']*df['incorrect']

  df['start_event']=(df['event_code']==2000).astype('int')
  df['uid']=df['start_event'].cumsum()
  df['end_event']=df['start_event'].shift(-1,fill_value=1)
  df['start_assessment']=(df['type']=='Assessment').astype(int)*(df['start_event']).astype(int)
  df['timestamp']=pd.to_datetime(df['timestamp'])
  #Sum of correct and incorrect for each event start and merge


  agg_df=df.groupby(['uid']).agg({'timestamp':['min','max'],'installation_id':'first','correct':'sum','incorrect':'sum'})
  agg_df.columns=['ts_min','ts_max','installation_id','total_correct','total_incorrect']
  agg_df['game_duration']=(agg_df['ts_min']-agg_df['ts_min'].shift(1)).dt.days*86400+(agg_df['ts_min']-agg_df['ts_min'].shift(1)).dt.seconds+(agg_df['ts_min']-agg_df['ts_min'].shift(1)).dt.microseconds/1e6
  agg_df=agg_df.reset_index()
  df=df.merge(agg_df[['total_correct','total_incorrect','uid','game_duration']],on='uid',how='left')
  assessments = ['Bird Measurer (Assessment)', 'Cart Balancer (Assessment)', 
                   'Cauldron Filler (Assessment)', 'Chest Sorter (Assessment)', 'Mushroom Sorter (Assessment)']

  #correct incorrect for each title

  df['super_token']=df['title']+df['event_code'].astype(str)+df['correct'].astype(str)+df['incorrect'].astype(str)

  assessments = ['Bird Measurer (Assessment)', 'Cart Balancer (Assessment)', 
                   'Cauldron Filler (Assessment)', 'Chest Sorter (Assessment)', 'Mushroom Sorter (Assessment)']
  assessment_fts=[]
  for title in assessments:
    df[title+'_correct'] = 0
    df[title+'_incorrect'] = 0
    assessment_fts+=[title+'_incorrect',title+'_correct']
    df.loc[df['title']==title,title+'_correct']=df.loc[df['title']==title].groupby('installation_id')['correct'].transform(pd.Series.cumsum)
    df.loc[df['title']==title,title+'_incorrect']=df.loc[df['title']==title].groupby('installation_id')['incorrect'].transform(pd.Series.cumsum)

  #inference change in installation id
  #metric point 
  df['metric_point']=df['start_assessment']*(df['correct']+df['incorrect']>0)
  df['metric_point_inference']=(df['installation_id'].shift(-1)!=df['installation_id']).astype(int)
  return df[assessment_fts+['installation_id','title','super_token','event_code','game_duration','total_correct','total_incorrect','uid','metric_point','metric_point_inference','correct','incorrect']]
def create_text_file(typ):
  df=load_csv(typ)

  df['event_idx']=1
  df['event_idx']=df.groupby(['installation_id'])['event_idx'].transform(pd.Series.cumsum)


  if typ=='train':
    lbl={}
    for en,sup in enumerate(df['super_token'].unique()):
      lbl[sup]=en+1
    with open('txt_labels.pickle','wb') as handle:
      pickle.dump(lbl,handle)
    
  else:
    with open('txt_labels.pickle','rb') as handle:
      lbl=pickle.load(handle)
  df['super_token']=df['super_token'].map(lambda x: lbl.get(x,0)).astype(np.int16)

  all_acts=[]
  for ix in df.loc[df['metric_point_inference']==1].index:
    ei=df.loc[ix].event_idx
    all_acts.append(df.loc[ix-ei+1:ix+1]['super_token'].values.tolist())
  np.save(typ+'_all',np.asarray(all_acts))



  acts_before=[]
  for ix in df.loc[df['metric_point']==1].index:
    ei=df.loc[ix].event_idx
    acts_before.append(df.loc[ix-ei+1:ix+1]['super_token'].values.tolist())
  np.save(typ+'_upto',np.asarray(acts_before))

  assessments = ['Bird Measurer (Assessment)', 'Cart Balancer (Assessment)', 
                   'Cauldron Filler (Assessment)', 'Chest Sorter (Assessment)', 'Mushroom Sorter (Assessment)']


  for a in assessments:
    feat1=a+'_correct'
    feat2=a+'_incorrect'
    feat3=a+'_rate'
    df.loc[(df[feat1]+df[feat2])>0,feat3]=df.loc[(df[feat1]+df[feat2])>0,feat1]/(df.loc[(df[feat1]+df[feat2])>0,feat1]+df.loc[(df[feat1]+df[feat2])>0,feat2])

    
  

  df['labels']=0
  df.loc[(df['total_incorrect']>2)&(df['total_correct']>0),'labels']=1
  df.loc[(df['total_incorrect']==1)&(df['total_correct']>0),'labels']=2
  df.loc[(df['total_incorrect']==0)&(df['total_correct']>0),'labels']=3
  df['total_attempts']=df['total_correct']+df['total_incorrect']
  count=pd.read_csv('count.csv')
  df=df.merge(count,on='uid',how='left').drop(['uid'],1)
  df['all_attempts']=df['correct']+df['incorrect']
  df.to_csv(typ+'.csv',index=False)

Overwriting preprocessing.py


In [137]:
%%writefile train_preprocessing.py
from preprocessing import create_text_file
create_text_file('train')

Overwriting train_preprocessing.py


In [138]:
%%writefile test_preprocessing.py
from preprocessing import create_text_file
create_text_file('test')

Overwriting test_preprocessing.py


In [139]:
!python train_preprocessing.py
!python test_preprocessing.py

In [140]:
texts=np.load('train_all.npy',allow_pickle=True).tolist()
for i,t in enumerate(texts):
  texts[i] = ' '.join(['q' + str(q) for q in t])
texts  

['q1 q2 q3 q4 q5 q6 q6 q6 q6 q6 q6 q6 q6 q6 q6 q6 q6 q6 q4 q6 q6 q6 q6 q5 q6 q6 q6 q6 q6 q6 q4 q6 q5 q6 q6 q6 q6 q6 q6 q6 q6 q4 q6 q5 q6 q6 q6 q7 q4 q5 q6 q6 q6 q6 q6 q6 q8 q9 q6 q8 q9 q6 q6 q6 q6 q8 q9 q6 q6 q6 q8 q9 q8 q9 q8 q9 q6 q8 q9 q6 q8 q9 q8 q9 q8 q9 q8 q9 q6 q8 q9 q6 q8 q9 q8 q9 q8 q9 q8 q9 q8 q9 q8 q9 q8 q9 q8 q9 q8 q9 q8 q9 q8 q9 q8 q9 q8 q9 q8 q9 q8 q9 q6 q6 q8 q9 q6 q6 q6 q8 q9 q8 q9 q8 q9 q8 q9 q6 q6 q6 q8 q9 q6 q8 q9 q8 q9 q8 q9 q6 q6 q6 q8 q10 q8 q10 q6 q8 q4 q10 q5 q8 q9 q6 q8 q9 q8 q11 q4 q5 q6 q6 q6 q4 q6 q8 q5 q4 q9 q5 q6 q8 q4 q10 q8 q10 q6 q5 q6 q8 q10 q8 q11 q4 q8 q5 q8 q11 q4 q6 q5 q6 q8 q10 q6 q8 q4 q10 q5 q8 q10 q4 q8 q5 q11 q4 q5 q6 q8 q4 q10 q5 q6 q8 q4 q10 q8 q5 q11 q4 q5 q6 q8 q9 q8 q10 q4 q8 q5 q11 q4 q5 q6 q6 q8 q9 q8 q9 q8 q6 q9 q6 q8 q9 q6 q8 q10 q8 q10 q8 q4 q10 q8 q5 q11 q4 q5 q8 q9 q3 q4 q6 q6 q6 q6 q6 q6 q5 q8 q10 q6 q8 q4 q10 q5 q6 q8 q4 q10 q8 q5 q11 q4 q5 q4 q6 q5 q8 q10 q4 q5 q4 q8 q5 q8 q4 q10 q8 q5 q11 q4 q8 q5 q9 q6 q8 q10 q4 q8 q5 q4 q11 q

In [149]:
%%writefile tfidf.py
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import gc
from scipy.sparse import save_npz
texts=np.load('train_all.npy',allow_pickle=True).tolist()+np.load('test_all.npy',allow_pickle=True).tolist()
for i,t in enumerate(texts):
  texts[i]=' '.join(['q'+str(q) for q in t])
vectorizer=TfidfVectorizer( input='content', encoding='utf-8', decode_error='strict', strip_accents=None,
                             lowercase=False, preprocessor=None, tokenizer=None, analyzer='word',
                             stop_words=None, token_pattern='\S+', ngram_range=(1, 3), max_df=0.99, min_df=100,
                             max_features=None, vocabulary=None, binary=False,
                              norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=False)
vectorizer.fit(texts)
del texts
gc.collect()
for f in ['test_upto.npy','train_upto.npy','test_all.npy']:
  texts=np.load(f,allow_pickle=True).tolist()
  for i,t in enumerate(texts):
    texts[i]=''.join(['q'+str(q) for q in t])
  texts=vectorizer.transform(texts)
  save_npz(f,texts)
  del texts 
  gc.collect()

Writing tfidf.py


In [150]:
%%time
!python tfidf.py

Traceback (most recent call last):
  File "tfidf.py", line 20, in <module>
    texts=vectorizer.transform(texts)
  File "/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py", line 1899, in transform
    return self._tfidf.transform(X, copy=False)
  File "/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py", line 1490, in transform
    X = check_array(X, accept_sparse='csr', dtype=FLOAT_DTYPES, copy=copy)
  File "/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py", line 586, in check_array
    context))
ValueError: Found array with 0 sample(s) (shape=(0, 6)) while a minimum of 1 is required.
CPU times: user 20.8 ms, sys: 10.4 ms, total: 31.3 ms
Wall time: 2.87 s


In [ ]:
import numpy as np
import random
import tensorflow as tf
import scipy
seed=239
tf.rando,.set_seed(seed)
random.seed(seed)
np.random.seed(seed)


q_train=scipy.sparse.load('train_upto.npy')
q_test=scipy.sparse.load('test_upto.npy')

In [ ]:
df[['installation_id','metric_point_inference']]

,installation_id,metric_point_inference
0,0001e90f,0
1,0001e90f,0
2,0001e90f,0
3,0001e90f,0
4,0001e90f,0
5,0001e90f,0
6,0001e90f,0
7,0001e90f,0
8,0001e90f,0
9,0001e90f,1


In [ ]:
lbl=pd.read_csv('train_labels.csv.zip')
lbl.head()

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.0,0
2,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
3,9501794defd84e4d,0006a69f,Mushroom Sorter (Assessment),1,1,0.5,2
4,a9ef3ecb3d1acc6a,0006a69f,Bird Measurer (Assessment),1,0,1.0,3
